In [548]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import random 
import math

In [549]:
data = pd.read_excel("Shipping Data.xlsx")
data

,SJ_NUMBER,SJ_LOADING_DATE,CARRIER_CODE,CARRIER_DESCRIPTION,WEIGHT_CAPACITY,VOLUME_CAPACITY,SHIP_TO_ORG_ID,LONGITUDE,LATITUDE,SHIPPED_QUANTITY,TOTAL_WEIGHT,TOTAL_VOLUME
0,SJ-202404071,2024-05-20 10:28:00,CB02,COLT L-300,1160,4.70,3367684,-6.135728,106.790511,24,7.000008,0.008687
1,SJ-202404853,2024-06-14 10:27:21,CB02,COLT L-300,1160,4.70,3367732,-6.144215,106.796802,3,0.699290,0.000980
2,SJ-202403866,2024-05-13 15:59:08,CB01,MOTOR BOX,50,0.14,1167762,-6.167749,106.872207,1,0.143000,0.000620
3,SJ-202403699,2024-05-07 16:26:40,CB01,MOTOR BOX,50,0.14,1167762,-6.167749,106.872207,15,0.541667,0.004983
4,SJ-202405662,2024-07-10 15:36:31,CB01,MOTOR BOX,50,0.14,1167762,-6.167749,106.872207,80,2.066667,0.007146
...,...,...,...,...,...,...,...,...,...,...,...,...
28971,SJ-202405310,2024-06-29 11:01:50,CB02,COLT L-300,1160,4.70,4750208,-6.193153,106.803224,24,7.000008,0.008687
28972,SJ-202405560,2024-07-08 09:37:50,CB03,COLT ENGKEL 3T/7M3,2850,7.68,4751111,-6.304409,106.862343,6,2.810000,0.011513
28973,SJ-202405418,2024-07-03 10:45:39,CB01,MOTOR BOX,50,0.14,4751967,NaN,NaN,6,1.193000,0.003419
28974,SJ-202405777,2024-07-13 13:30:57,CB06,MINI BUS,700,4.30,4758127,NaN,NaN,1212,540.800000,1.723297


In [550]:
data = data.drop(['SJ_NUMBER', 'SHIP_TO_ORG_ID', 'SHIPPED_QUANTITY'], axis=1)
data

,SJ_LOADING_DATE,CARRIER_CODE,CARRIER_DESCRIPTION,WEIGHT_CAPACITY,VOLUME_CAPACITY,LONGITUDE,LATITUDE,TOTAL_WEIGHT,TOTAL_VOLUME
0,2024-05-20 10:28:00,CB02,COLT L-300,1160,4.70,-6.135728,106.790511,7.000008,0.008687
1,2024-06-14 10:27:21,CB02,COLT L-300,1160,4.70,-6.144215,106.796802,0.699290,0.000980
2,2024-05-13 15:59:08,CB01,MOTOR BOX,50,0.14,-6.167749,106.872207,0.143000,0.000620
3,2024-05-07 16:26:40,CB01,MOTOR BOX,50,0.14,-6.167749,106.872207,0.541667,0.004983
4,2024-07-10 15:36:31,CB01,MOTOR BOX,50,0.14,-6.167749,106.872207,2.066667,0.007146
...,...,...,...,...,...,...,...,...,...
28971,2024-06-29 11:01:50,CB02,COLT L-300,1160,4.70,-6.193153,106.803224,7.000008,0.008687
28972,2024-07-08 09:37:50,CB03,COLT ENGKEL 3T/7M3,2850,7.68,-6.304409,106.862343,2.810000,0.011513
28973,2024-07-03 10:45:39,CB01,MOTOR BOX,50,0.14,NaN,NaN,1.193000,0.003419
28974,2024-07-13 13:30:57,CB06,MINI BUS,700,4.30,NaN,NaN,540.800000,1.723297


In [551]:
data['SJ_LOADING_DATE'] = pd.to_datetime(data['SJ_LOADING_DATE'])

In [552]:
tanggal = '2024-05-02'
waktu_mulai = '07:00:00'
waktu_akhir = '09:00:00'

start_datetime = pd.to_datetime(f"{tanggal} {waktu_mulai}")
end_datetime = pd.to_datetime(f"{tanggal} {waktu_akhir}")

In [553]:
data_used = data[(data['SJ_LOADING_DATE'] >= start_datetime) & (data['SJ_LOADING_DATE'] <= end_datetime)]
data_used = data_used.drop(['SJ_LOADING_DATE', 'CARRIER_CODE', 'CARRIER_DESCRIPTION', 'WEIGHT_CAPACITY', 'VOLUME_CAPACITY'], axis=1).reset_index(drop=True)

depo_point = pd.DataFrame({'LONGITUDE': [-6.181794196399728], 'LATITUDE': [106.91137272072216], 'TOTAL_WEIGHT': [0] , 'TOTAL_VOLUME': [0]})

data_used = pd.concat([depo_point, data_used], ignore_index=True)
data_used

,LONGITUDE,LATITUDE,TOTAL_WEIGHT,TOTAL_VOLUME
0,-6.181794,106.911373,0.000000,0.000000
1,-6.254061,106.790543,5.888000,0.042966
2,-6.177427,106.894859,0.304944,0.001059
3,-6.137841,106.870198,3.065689,0.003800
4,-6.172385,106.898890,0.504562,0.001938
5,-6.147109,106.907927,1.008000,0.002227
6,-6.158287,106.870624,102.090000,0.438742
7,-6.241767,106.839168,68.503198,0.478626
8,-6.227326,106.804837,111.918233,0.524385
9,NaN,NaN,175.720424,0.563014


In [554]:
longlat = data_used.drop(['TOTAL_WEIGHT', 'TOTAL_VOLUME'], axis=1)
longlat

,LONGITUDE,LATITUDE
0,-6.181794,106.911373
1,-6.254061,106.790543
2,-6.177427,106.894859
3,-6.137841,106.870198
4,-6.172385,106.898890
5,-6.147109,106.907927
6,-6.158287,106.870624
7,-6.241767,106.839168
8,-6.227326,106.804837
9,NaN,NaN


In [555]:
unique_carriers = data.drop_duplicates(subset=['CARRIER_CODE'])

carrier = unique_carriers[['CARRIER_CODE', 'CARRIER_DESCRIPTION', 'WEIGHT_CAPACITY', 'VOLUME_CAPACITY']].reset_index(drop=True)
carrier

,CARRIER_CODE,CARRIER_DESCRIPTION,WEIGHT_CAPACITY,VOLUME_CAPACITY
0,CB02,COLT L-300,1160,4.70
1,CB01,MOTOR BOX,50,0.14
2,CB04,COLT 6 BAN,4000,12.00
3,CB06,MINI BUS,700,4.30
4,CB03,COLT ENGKEL 3T/7M3,2850,7.68
5,BU1445,BUILT UP 14T/45M3,14000,45.00
6,CBXX,SJAT,145000,650.00


In [556]:
if data_used['TOTAL_WEIGHT'].sum() > 22000:
  carrier_used = carrier
elif data_used['TOTAL_WEIGHT'].sum() > 10000:
  carrier_used = carrier.drop(6)
elif data_used['TOTAL_WEIGHT'].sum() > 4000:
  carrier_used = carrier.drop(5)
elif data_used['TOTAL_WEIGHT'].sum() > 1000:
  carrier_used = carrier.drop([2, 5, 6])
else:
  carrier_used = carrier.drop([2, 3, 5, 6])

In [557]:
carrier_used = carrier_used.reset_index(drop=True)

In [558]:
num_carriers = len(carrier_used)
num_locations = len(longlat)

def haversine(lon1, lat1, lon2, lat2):
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2.0)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def calculate_haversine_distance_matrix(haversine, longlat):
    num_locations = len(longlat)
    distance_matrix = np.zeros((num_locations, num_locations))
    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                distance = haversine(
                    longlat.loc[i, 'LONGITUDE'],
                    longlat.loc[i, 'LATITUDE'],
                    longlat.loc[j, 'LONGITUDE'],
                    longlat.loc[j, 'LATITUDE']
                )
                distance_matrix[i][j] = distance
            else:
                distance_matrix[i][j] = 0
    return distance_matrix

In [559]:
distance_matrix = calculate_haversine_distance_matrix(haversine, longlat)
distance_matrix = np.nan_to_num(distance_matrix, nan=0.0, posinf=1e6, neginf=1e6)

distance_matrix

array([[ 0.        , 13.63605217,  1.84161796,  4.79354103,  1.42092145,
         1.18543182,  4.59427192,  8.2589012 , 11.93689589,  0.        ],
       [13.63605217,  0.        , 11.85927849,  9.61514039, 12.33174029,
        13.49993935,  9.42332968,  5.42123951,  1.80669658,  0.        ],
       [ 1.84161796, 11.85927849,  0.        ,  3.02550223,  0.47692476,
         1.75266148,  2.76483881,  6.53128098, 10.13843667,  0.        ],
       [ 4.79354103,  9.61514039,  3.02550223,  0.        ,  3.37984849,
         4.20587648,  0.66145479,  4.80958211,  7.81848894,  0.        ],
       [ 1.42092145, 12.33174029,  0.47692476,  3.37984849,  0.        ,
         1.2951485 ,  3.1759021 ,  7.00805765, 10.60717647,  0.        ],
       [ 1.18543182, 13.49993935,  1.75266148,  4.20587648,  1.2951485 ,
         0.        ,  4.16356839,  8.2334417 , 11.75124844,  0.        ],
       [ 4.59427192,  9.42332968,  2.76483881,  0.66145479,  3.1759021 ,
         4.16356839,  0.        ,  4.41343216

In [560]:
num_locations = len(distance_matrix)
num_carriers = len(carrier_used)

# Gurobi Model
model = gp.Model("CVRP_ILP")

# Variabel Keputusan
x = model.addVars(
    num_locations, num_locations, num_carriers, vtype=GRB.BINARY, name="x"
)
u = model.addVars(
    num_locations, num_carriers, vtype=GRB.CONTINUOUS, name="u"
)

# Fungsi Objektif: Minimalkan total jarak
model.setObjective(
    quicksum(
        distance_matrix[i][j] * x[i, j, k]
        for i in range(num_locations)
        for j in range(num_locations)
        for k in range(num_carriers)
    ),
    GRB.MINIMIZE,
)

# 1. Setiap lokasi (kecuali depot) harus dilayani tepat satu kali
for j in range(1, num_locations):  # Tidak termasuk depot
    model.addConstr(
        quicksum(x[i, j, k] for i in range(num_locations) for k in range(num_carriers))
        == 1,
        name=f"location_served_once_{j}",
    )

# 2. Kendaraan yang masuk sama dengan kendaraan yang keluar untuk setiap lokasi
for k in range(num_carriers):
    for i in range(num_locations):
        model.addConstr(
            quicksum(x[i, j, k] for j in range(num_locations)) ==
            quicksum(x[j, i, k] for j in range(num_locations)),
            name=f"flow_balance_{i}_{k}"
        )

# 3. Kendaraan harus kembali ke depot
for k in range(num_carriers):
    model.addConstr(quicksum(x[0, j, k] for j in range(1, num_locations)) == 1)
    model.addConstr(quicksum(x[i, 0, k] for i in range(1, num_locations)) == 1)

# 4. Kapasitas berat kendaraan tidak boleh terlampaui
for k in range(num_carriers):
    model.addConstr(
        quicksum(
            data_used.iloc[i]["TOTAL_WEIGHT"] *
            quicksum(x[i, j, k] for j in range(num_locations))
            for i in range(1, num_locations)
        ) <= carrier_used.iloc[k]["WEIGHT_CAPACITY"],
        name=f"weight_capacity_{k}",
    )

# 5. Kapasitas volume kendaraan tidak boleh terlampaui
for k in range(num_carriers):
    model.addConstr(
        quicksum(
            data_used.iloc[i]["TOTAL_VOLUME"] *
            quicksum(x[i, j, k] for j in range(num_locations))
            for i in range(1, num_locations)
        ) <= carrier_used.iloc[k]["VOLUME_CAPACITY"],
        name=f"volume_capacity_{k}",
    )

# 6. Subtour elimination (MTZ constraints)
for k in range(num_carriers):
    for i in range(1, num_locations):  # Lokasi non-depot
        for j in range(1, num_locations):  # Lokasi non-depot
            if i != j:
                model.addConstr(
                    u[i, k] - u[j, k] + (num_locations - 1) * x[i, j, k] <= num_locations - 2,
                    name=f"mtz_{i}_{j}_{k}",
                )
    # Awal nilai u untuk depot harus 0
    model.addConstr(u[0, k] == 0, name=f"depot_u_{k}")

# Constraint: Tidak ada loop di lokasi yang sama
for k in range(num_carriers):
    for i in range(num_locations):
        model.addConstr(x[i, i, k] == 0, name=f"no_self_loop_{i}_{k}")



# Solve Model
model.optimize()

# Output Solusi
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution Found!")
    print(f"Total Distance: {model.objVal:.2f}")
    
    for k in range(num_carriers):
        carrier_code = carrier_used.iloc[k]['CARRIER_CODE']
        route = [0]  # Mulai dari depot
        total_distance = 0
        current_location = 0
        visited = set()  # Menyimpan lokasi yang sudah dikunjungi
        visited.add(0)  # Depot sudah dikunjungi

        while True:
            next_location = None
            for j in range(num_locations):
                if j not in visited and x[current_location, j, k].X > 0.5:
                    next_location = j
                    total_distance += distance_matrix[current_location][j]
                    break

            if next_location is None:  # Tidak ada lokasi lagi yang bisa dikunjungi
                route.append(0)  # Kembali ke depot
                total_distance += distance_matrix[current_location][0]
                break

            route.append(next_location)
            visited.add(next_location)
            current_location = next_location  # Update lokasi saat ini

        print(f"\nCarrier Code: {carrier_code}")
        print(f"Route: {route}")
        print(f"Total Distance: {total_distance:.2f}")


else:
    print("\nNo Optimal Solution Found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 300 rows, 330 columns and 2085 nonzeros
Model fingerprint: 0x8acfe4c4
Variable types: 30 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e-03, 2e+02]
  Objective range  [5e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 3e+03]
Presolve removed 85 rows and 96 columns
Presolve time: 0.01s
Presolved: 215 rows, 234 columns, 1321 nonzeros
Variable types: 24 continuous, 210 integer (210 binary)

Root relaxation: objective 1.383225e+01, 81 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.83225    0   12         

In [326]:
carrier_used

,CARRIER_CODE,CARRIER_DESCRIPTION,WEIGHT_CAPACITY,VOLUME_CAPACITY
0,CB02,COLT L-300,1160,4.70
1,CB01,MOTOR BOX,50,0.14
2,CB03,COLT ENGKEL 3T/7M3,2850,7.68
